In [ ]:
''' 
packages
'''


import pandas as pd 
import ast
from googletrans import Translator
from multiprocessing import Pool, Manager
import numpy as np
from collections import Counter




In [ ]:

''' 

Translation of transcibing file

'''


df_transciption = pd.read_csv('data/complete_data_transcribing.csv')
df_transciption = df_transciption[df_transciption.sentences != 'failed']
df_transciption['ids'] = df_transciption['id'] 
df_transciption['comp'] = np.arange(len(df_transciption))
del df_transciption['Unnamed: 0']
translator = Translator()
def update_data_with_trans(A):
    d = A[0]
    i = A[1]
    extracted =  df_transciption[df_transciption.comp == i]['sentences'].values[0]
    d[i] = translator.translate(extracted, dest='en').text

manager = Manager()
translated_dict = manager.dict()
with Pool(15) as p:
    p.map(update_data_with_trans, [[translated_dict,i] for i in np.arange(len(df_transciption))])
df_transciption['translated'] = [translated_dict[i] for i in np.arange(len(df_transciption))]
del df_transciption['comp']
del df_transciption['ids']
df_transciption.to_csv('data/complete_data_transcribing_translated.csv', index=False)


In [ ]:
'''
Translation of attributes file 

'''



df_attributes = pd.read_csv('data/videos_alphapolitica_complete_info_ids.csv')
df_attributes = pd.read_csv('data/videos_alphapolitica_complete_info_ids.csv')
lis_tag = [ast.literal_eval(i) for i in df_attributes['tags']]
lis_tags = [j for i in lis_tag for j in i]
dict_counter = dict(Counter(lis_tags))
df_counter = pd.DataFrame()
df_counter['tags'] = dict_counter.keys()
df_counter['counter'] = dict_counter.values()
df_counter = df_counter.sort_values(by='counter', ascending= False)
stopwords = [i.lower() for i in df_counter[df_counter.counter > 7000]['tags'].values]
to_drop = ['today ap news', 'today telugu news','tv5 telugu live','ap news live','ap political news']
stopwords = stopwords + to_drop
df_attributes['tags'] = [ast.literal_eval(i) for i in df_attributes['tags']]
df_attributes['tags_updated'] = [list(set([j.lower() for j in i ])-set(stopwords)) for i in  df_attributes['tags'].values]
df_attributes['tags'] = [' '.join(i) for i in df_attributes['tags_updated']]
del df_attributes['tags_updated']

del df_attributes['Unnamed: 0']
df_attributes = df_attributes.dropna(subset=['title'])
df_attributes['tags'] = [' '.join(ast.literal_eval(i)) for i in df_attributes['tags']]
df_attributes = df_attributes[['ids','title','tags','upload_date','duration','comment_count','view_count','uploader']]
df_attributes['comp'] = np.arange(len(df_attributes))
translator = Translator()
def update_data_with_trans(A):
    d = A[0]
    i = A[1]
    extracted =  df_attributes[df_attributes.comp == i]['title'].values[0]
    d[i] = translator.translate(extracted, dest='en').text

manager = Manager()
translated_dict = manager.dict()
with Pool(15) as p:
    p.map(update_data_with_trans, [[translated_dict,i] for i in np.arange(len(df_attributes))])
df_attributes['title_translated'] = [translated_dict[i] for i in np.arange(len(df_attributes))]
del df_attributes['comp']
df_attributes.to_csv('data/videos_alphapolitica_complete_info_ids_translated.csv', index= False)


In [94]:
df_attributes = pd.read_csv('data/videos_alphapolitica_complete_info_ids_translated.csv')


In [105]:
df_attributes['tags'] = df_attributes['tags'].fillna('')

In [109]:
df_attributes

,ids,title,tags,upload_date,duration,comment_count,view_count,uploader,title_translated
0,vnPABjrcdwE,Morning Breaking : స్టూడెంట్ పై ప్రొఫెసర్ అత్య...,,20221203,58,NaN,2044,TV9 Telugu Live,Morning Breaking : Professor rape attempt on s...
1,hLQWlhQRP20,సోము వీర్రాజు బీజేపీని టీడీపీకి అద్దెకు ఇచ్చాడ...,kodali nani about grama sachivalayam ap capita...,20220128,1289,29.0,17058,Sakshi TV Live,Somu Veerraju rents BJP to TDP Kodali Nani Pre...
2,Rtor62OCiys,AP Second Phase Panchayat Election Results | R...,ap local body elections results ap local body ...,20210214,221,96.0,144107,Sakshi TV Live,AP Second Phase Panchayat Election Results | R...
3,ejUJYTFmDy4,PM Modi Speech LIVE || Modi Celebrates Diwali ...,modi pm modi live pm modi diwali jawans modi s...,20221023,3683,NaN,6365,TV5 News,PM Modi Speech LIVE || Modi Celebrates Diwali ...
4,8GJeex_Y81o,TRS MLA Pilot Rohith Reddy And Ramachandra Bha...,ramachandra bharati bjp ramachandra bharathi p...,20221028,718,1.0,3197,TV5 News,TRS MLA Pilot Rohith Reddy And Ramachandra Bha...
...,...,...,...,...,...,...,...,...,...
99672,SwN0RdSmf6U,షర్మిల వెనుక బీజేపీ ఉంది | TRS Leaders strong ...,,20221201,128,8.0,4265,TV9 Telugu Live,BJP is behind Sharmila TRS Leaders strong coun...
99673,YWR1XYLk7CQ,నెల్లూరులో విషజ్వరాల కలకలం.. ఒకేరోజు ముగ్గురు ...,,20221025,372,NaN,1093,ABN Telugu,Poisonous fever in Nellore.. Three died in one...
99674,QaKljaCyuS0,PM Narendra Modi Convoy in Hyderabad | Sakshi ...,modi hyderabad tour modi begumpet speech hyder...,20220526,68,31.0,74748,Sakshi TV Live,PM Narendra Modi Convoy in Hyderabad | Sakshi ...
99675,KTN5DOB4v9g,Minister Talasani Srinivas Participated In Hol...,,20230307,100,NaN,406,TV5 News,Minister Talasani Srinivas Participated In Hol...


In [87]:
''' 
data merging

'''

df_transciption = pd.read_csv('data/complete_data_transcribing_translated.csv')
df_attributes = pd.read_csv('data/videos_alphapolitica_complete_info_ids_translated.csv')
df_attributes_merged = df_attributes.merge(df_transciption, on='ids', how='outer')
df_attributes_merged = df_attributes_merged.dropna(subset = ['title'])


In [91]:
df_attributes_merged


,ids,title,tags,upload_date,duration,comment_count,view_count,uploader,title_translated,sentences,translated
0,vnPABjrcdwE,Morning Breaking : స్టూడెంట్ పై ప్రొఫెసర్ అత్య...,NaN,20221203.0,58.0,NaN,2044.0,TV9 Telugu Live,Morning Breaking : Professor rape attempt on s...,హైదరాబాద్ సెల్ యూనివర్సిటీ జరిగే దారుణానికి సం...,He announced a story in the brain about the at...
1,hLQWlhQRP20,సోము వీర్రాజు బీజేపీని టీడీపీకి అద్దెకు ఇచ్చాడ...,kodali nani about grama sachivalayam ap capita...,20220128.0,1289.0,29.0,17058.0,Sakshi TV Live,Somu Veerraju rents BJP to TDP Kodali Nani Pre...,NaN,NaN
2,Rtor62OCiys,AP Second Phase Panchayat Election Results | R...,ap local body elections results ap local body ...,20210214.0,221.0,96.0,144107.0,Sakshi TV Live,AP Second Phase Panchayat Election Results | R...,అనంతపురం జిల్లాలో మాజీ మంత్రి పరిటాల సునీత కంచ...,Former minister Paritala Sunitha Kanchota brok...
3,ejUJYTFmDy4,PM Modi Speech LIVE || Modi Celebrates Diwali ...,modi pm modi live pm modi diwali jawans modi s...,20221023.0,3683.0,NaN,6365.0,TV5 News,PM Modi Speech LIVE || Modi Celebrates Diwali ...,NaN,NaN
4,8GJeex_Y81o,TRS MLA Pilot Rohith Reddy And Ramachandra Bha...,ramachandra bharati bjp ramachandra bharathi p...,20221028.0,718.0,1.0,3197.0,TV5 News,TRS MLA Pilot Rohith Reddy And Ramachandra Bha...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
99677,uNETz4Qdv6s,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,మగిని ప్రభుత్వ ఆసుపత్రి దగ్గర ఉన్నారు పిలుపు అ...,మగిని ప్రభుత్వ ఆసుపత్రి దగ్గర ఉన్నారు పిలుపు అ...
99678,ZFZ2JobOFWk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,పౌరసత్వ సవరణ చట్టానికి వ్యతిరేకంగా దేశవ్యాప్తం...,Protests continue across the country against t...
99679,m6_YF4K8g18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,నువ్వు డార్లింగ్ అంటే దెయ్యాల కూడా దేవతలుగా మా...,"You darling, even demons can become gods. Prab..."
99680,jjAHxtgbWE4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,కార్ణంగా భారత్‌లో తొలి మరణం నమోదైంది కర్ణాటకలో...,The first death was recorded in India due to t...


In [83]:
result

'telugu telugu news political updates elections 2024 breaking news today breaking news telugu latest news telugu telugu varthalu telanagna news telangana politics ap news ap politics'

In [ ]:
topics = ['chandhra babu','YCP', 'jagan', 'TDP','murder',
 'vivek', 'roads', 'capital', 'atrocities','volunteer', 'amma vadi',
  'welfare', 'schemes', 'lokesh', 'pattabi', 'vijayawada', 'visaka',
   'jogi ramesh','vamsi','kodali','nani', 'Education','Agriculture',
   'farmers','Corruption','investment','Budget','SC/ST','Polavaram',
   'irrigation','YSRCP','Telugu Desam Party','Vidya Deevena','education',
   'governance', ]


In [ ]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device = 'cuda:0')


In [ ]:

topics = ['chandhra babu','YCP', 'jagan', 'TDP','murder', 'vivek', 'roads', 'capital', 'atrocities','volunteer', 'amma vadi', 'welfare', 'schemes', 'lokesh', 'pattabi', 'vijayawada', 'visaka', 'jogi ramesh','vamsi','kodali','nani']
predictions = {}
for topic in topics:
    predictions[topic] = []
    sentences = df_one_shot['to_pred'].values
    for sent in tqdm(sentences):
        out = classifier(sent, ['chandhra babu'])
        predictions[topic].append(out['scores'][0])

In [ ]:
sequence_to_classify

In [ ]:
df_para['para_phrased'].values[0]

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

device = "cpu"

tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")

model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to(device)

def paraphrase(
    question,
    num_beams=5,
    num_beam_groups=5,
    num_return_sequences=5,
    repetition_penalty=10.0,
    diversity_penalty=3.0,
    no_repeat_ngram_size=2,
    temperature=0.7,
    max_length=128
):
    input_ids = tokenizer(
        f'paraphrase: {question}',
        return_tensors="pt", padding="longest",
        max_length=max_length,
        truncation=True,
    ).input_ids
    
    outputs = model.generate(
        input_ids, temperature=temperature, repetition_penalty=repetition_penalty,
        num_return_sequences=num_return_sequences, no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams, num_beam_groups=num_beam_groups,
        max_length=max_length, diversity_penalty=diversity_penalty
    )

    res = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return res

In [ ]:
from tqdm import tqdm
para_phrased = []
for i in tqdm(df_trans['translated'].values):
    para_phrased.append(paraphrase(i))

In [ ]:
df_trans['translated'].values[3]

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import sys
from tqdm import tqdm
import pandas as pd

device = "cpu"
tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")
model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to(device)

start = int(sys.argv[1])
end = int(sys.argv[2])
print('start', sys.argv[1])
print('end', sys.argv[2])

def paraphrase(
    question,
    num_beams=5,
    num_beam_groups=5,
    num_return_sequences=5,
    repetition_penalty=10.0,
    diversity_penalty=3.0,
    no_repeat_ngram_size=2,
    temperature=0.7,
    max_length=128
):
    input_ids = tokenizer(
        f'paraphrase: {question}',
        return_tensors="pt", padding="longest",
        max_length=max_length,
        truncation=True,
    ).input_ids
    outputs = model.generate(
        input_ids, temperature=temperature, repetition_penalty=repetition_penalty,
        num_return_sequences=num_return_sequences, no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams, num_beam_groups=num_beam_groups,
        max_length=max_length, diversity_penalty=diversity_penalty
    )
    res = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return res

df_trans = pd.read_csv('../data/kowshik_file_translated.csv')
del df_trans['Unnamed: 0']
del df_trans['sno']
para_phrased = []
for i in tqdm(df_trans['translated'].values):
    try:
        para_phrased.append(paraphrase(i))
    except:
        para_phrased.append(['None'])
df_trans['para_phrased'] = para_phrased
df_trans.to_csv('../data/kowshik_file_translated_para_phrased_' + str(start) +'_' +str(end) + '.csv')

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

sentences = ['This is an example sentence', 'Each sentence is converted']
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
model = AutoModel.from_pretrained('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
with torch.no_grad():
    model_output = model(**encoded_input)
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
print("Sentence embeddings:")
print(sentence_embeddings)


In [ ]:
1100*5.5*2000 + 1500000

In [ ]:
df = pd.read_csv('data/kowshik_file_translated_para_phrased_1_10.csv')

In [ ]:
import pandas as pd